In [133]:
import numpy as np
import torch
from torch.utils.data import Dataset

index_map = {
    0: "AGG",
    1: "BIL",
    2: "BND",
    3: "EDV",
    4: "IEF",
    5: "IEI",
    6: "SHV",
    7: "SHY",
    8: "TLH",
    9: "TLT",
    10: "VGIT"
}

class BondDataset(Dataset):
    def __init__(self, lag, target=0, file_path="bond_etfs.csv", transform=None, target_transform=None):
        self.lag = lag
        self.target = target
        self.records = np.genfromtxt(
            file_path, 
            delimiter=",", 
            skip_header=1,
            usecols = (1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12)
        ).astype(np.float32)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.records) - self.lag

    def __getitem__(self, idx):
        records = self.records[idx:idx + self.lag + 1]
        x = torch.from_numpy(records[:-1])
        y = torch.from_numpy(np.atleast_1d(records[-1][self.target] - records[-2][self.target]))
        if self.transform:
            x = self.transform(x)
        if self.target_transform:
            y = self.target_transform(y)
        return x, y



In [396]:
from torch.utils.data import DataLoader
from torch import nn
from torchvision import datasets, transforms

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

full_dataset = BondDataset(lag=15, target=1)
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [0.8, 0.2])

Using cpu device


In [397]:
class NeuralNetwork(nn.Module):
    def __init__(self, lag):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(lag*11, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork(full_dataset.lag).to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=165, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): ReLU()
    (8): Linear(in_features=64, out_features=32, bias=True)
    (9): ReLU()
    (10): Linear(in_features=32, out_features=1, bias=True)
  )
)


In [399]:
learning_rate = 1e-4
batch_size = 128
epochs = 5

# Initialize the loss function
loss_fn = nn.MSELoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=True)

In [400]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, MSE = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            MSE += (torch.pow((pred - y), 2)).sum().item()

    test_loss /= num_batches
    MSE /= size
    print(f"Test Error: \n MSE: {(MSE):>8f}, Avg loss: {test_loss:>8f} \n")

In [402]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.002183  [  128/ 2000]
loss: 0.000695  [ 1408/ 2000]
Test Error: 
 MSE: 0.003347, Avg loss: 0.003323 

Epoch 2
-------------------------------
loss: 0.002780  [  128/ 2000]
loss: 0.003179  [ 1408/ 2000]
Test Error: 
 MSE: 0.003333, Avg loss: 0.003351 

Epoch 3
-------------------------------
loss: 0.000838  [  128/ 2000]
loss: 0.004120  [ 1408/ 2000]
Test Error: 
 MSE: 0.003297, Avg loss: 0.003231 

Epoch 4
-------------------------------
loss: 0.000703  [  128/ 2000]
loss: 0.002241  [ 1408/ 2000]
Test Error: 
 MSE: 0.003257, Avg loss: 0.003228 

Epoch 5
-------------------------------
loss: 0.001673  [  128/ 2000]
loss: 0.001742  [ 1408/ 2000]
Test Error: 
 MSE: 0.003236, Avg loss: 0.003208 

Epoch 6
-------------------------------
loss: 0.000597  [  128/ 2000]
loss: 0.003915  [ 1408/ 2000]
Test Error: 
 MSE: 0.003193, Avg loss: 0.003212 

Epoch 7
-------------------------------
loss: 0.000901  [  128/ 2000]
loss: 0.004606  [ 1408/ 2000]


In [433]:
X, Y = next(iter(test_dataloader))
print(X[0])
print(Y[0])
print(model(X)[0])

tensor([[105.7600,  91.4800,  78.8300, 111.0200, 101.1900, 119.1600, 110.2900,
          83.1600, 129.1300, 118.6000,  62.1100],
        [105.2900,  91.5000,  78.4600, 109.2600, 100.6300, 118.8000, 110.3100,
          83.1100, 128.1200, 117.2500,  61.9100],
        [105.1700,  91.5100,  78.4000, 108.5500, 100.4200, 118.6800, 110.3000,
          83.1100, 127.7800, 116.8000,  61.8100],
        [105.0300,  91.5100,  78.3300, 107.8900, 100.3600, 118.7300, 110.3300,
          83.1300, 127.5600, 116.2100,  61.8200],
        [105.3000,  91.5200,  78.5400, 109.0500, 100.7500, 118.9600, 110.3300,
          83.1700, 128.2100, 117.2100,  61.9900],
        [105.3100,  91.5200,  78.5400, 109.1900, 100.8200, 118.9900, 110.3200,
          83.1700, 128.3100, 117.2700,  62.0000],
        [105.3100,  91.5200,  78.5600, 108.8400, 100.7800, 118.9800, 110.3200,
          83.1700, 128.2400, 117.0900,  62.0100],
        [105.6000,  91.5300,  78.8000, 109.9100, 101.2400, 119.2800, 110.3600,
          83.2400,